**Instituto de Informática - UFG**

Disciplina: Inteligência Computacional

Prof. Cedric Luiz de Carvalho

Aluna(o): 
Arthur Gomes de Siqueira
Ricardo Ataide de Oliveira Junior


---


# Atividade Prática

* Deseja-se identificar a melhor rota para uma viagem rodoviária pelas capitais brasileiras. Para isso pode-se utilizar os dados obtidos do portal: [Itatrans](http://www.itatrans.com.br/distancia.html) e armazenados no arquivo **capitais.csv.**

* A porção inferior da planilha (células abaixo daquela com valor 0), traz à distância rodoviária entre as cidades. A porção superior, indica as distâncias aéreas entre as cidades.

* Considere que o mapa a seguir indique as conexões rodoviárias entre as capitais.  As conexões diretas entre as cidades estão indicadas no arquivo **conexCapitais.csv**

In [ ]:
!mkdir img/

In [ ]:
!mv mapa-brasil-estados-capitais.jpg img/

mv: cannot stat 'mapa-brasil-estados-capitais.jpg': No such file or directory


In [ ]:
from IPython.display import Image
Image("img/mapa-brasil-estados-capitais.jpg")

# Especificação:

  O trabalho deve ser desenvolvido em grupo de dois alunos, conforme especificado pelo professor, em comum acordo com a turma.
  Deve ser entregue um arquivo, no formato de *notebooks Jupyter*, contendo a solução desenvolvida pelo grupo. Esse arquivo deve conter uma descrição detalhada do modelo usado na solução do problema e sua codificação, em Python. Podem ser usadas ideias e implementações de terceiros, desde que sejam feitas as devidas citações das referências utilizadas. O trabalho deve seguir os seguintes passos:

*  Procure identificar soluções para o problema em questão (ou problemas parecidos) desenvolvidas por outras pessoas.
*  Defina um modelo para este problema a partir do estudo dos conceitos e das soluções propostas por outros.
*  Construa uma implementação computacional para a solução do problema, utilizando as seguintes estratégias:

     1.   Busca em largura
     2.   Busca em profundidade
     3.   Busca em profundidade limitada
     4.   Busca em aprofundamento iterativo
     5.   Busca de custo uniforme (considerar a distância rodoviária entre as cidades como o custo do caminho entre elas)
     
* Para cada estratégia implementada, meça o tempo gasto para encontrar a solução e o espaço de armazenamento utilizado (número de nós gerados).

* Faça uma avaliação criteriosa da solução produzida pelo grupo.


#Pontos a serem abordados: 

* Para cada capital é preciso identificar o melhor caminho para qualquer outra
* **As capitais que estão conectadas entre si já estão "resolvidas", no entanto também terão sua solução identificada através do algoritmo  de busca**
## Possível abordagem inicial 
* A tabela com as conexões será utilizada para construir a estrutura de dados que representa os caminhos possíveis.
* A tabela das distancias entre as cidades será utilizada para analise de melhor caminho (ou "otimicidade de caminho").



In [ ]:
import pandas as pd
import time


In [ ]:
#Fazendo upload dos arquivos
from google.colab import files
uploaded = files.upload()

Saving conexCapitais.csv to conexCapitais.csv
Saving distCapitais.csv to distCapitais.csv


In [ ]:
#Carregando os dados em dataframes
df_conexao = pd.read_csv("conexCapitais.csv")
df_conexao = df_conexao.rename(columns={'Unnamed: 0':'Cidade'}) # O dataframe pode ser carregado utilizando o index como o nome das cidades, ou um número automático. Foi utilizado a abordagem de um index automático
df_conexao = df_conexao.rename(columns={'R. Janeiro':'Rio de Janeiro'})

#Substituindo valores abreviados para facilitar nas buscas
df_conexao = df_conexao.replace({"Cidade": {"B. Horizonte" : "Belo Horizonte"}})
df_conexao = df_conexao.replace({"Cidade": {"C. Grande" : "Campo Grande"}})
df_conexao = df_conexao.replace({"Cidade": {"R. Janeiro": "Rio de Janeiro"}})

#Deletando a coluna e a linha com o nome Macapá, por ser irrelevante no problema
df_conexao = df_conexao.drop(columns="Macapá")
df_conexao = df_conexao.drop([12])
df_conexao = df_conexao.reset_index(drop=True)

#imprime o resultado da tabela conexCapitais.csv
df_conexao


,Cidade,Aracaju,Belém,Belo Horizonte,Boa Vista,Brasília,Campo Grande,Cuiabá,Curitiba,Florianópolis,Fortaleza,Goiânia,João Pessoa,Maceió,Manaus,Natal,Palmas,Porto Alegre,Porto Velho,Recife,Rio Branco,Rio de Janeiro,Salvador,São Luis,São Paulo,Teresina,Vitória
0,Aracaju,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
1,Belém,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
2,Belo Horizonte,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN
3,Boa Vista,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Brasília,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN
5,Campo Grande,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
6,Cuiabá,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Curitiba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
8,Florianópolis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Fortaleza,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN


In [ ]:
df_conexao.Aracaju[1] #teste

nan

In [ ]:
#Carregando os dados em dataframes
df_distancia = pd.read_csv("distCapitais.csv")
df_distancia = df_distancia.rename(columns={'Unnamed: 0':'Cidade'})
df_distancia = df_distancia.rename(columns={'R. Janeiro':'Rio de Janeiro'})

## Substituindo nomes abreviados para facilitar nas buscas
df_distancia = df_distancia.replace({"Cidade": {"B. Horizonte" : "Belo Horizonte"}})
df_distancia = df_distancia.replace({"Cidade": {"C. Grande" : "Campo Grande"}})
df_distancia = df_distancia.replace({"Cidade": {"R. Janeiro": "Rio de Janeiro"}})

#Deletando a coluna e a linha com o nome Macapá, por ser irrelevante no problema
df_distancia = df_distancia.drop(columns="Macapá")
df_distancia = df_distancia.drop([12])
df_distancia = df_distancia.reset_index(drop=True)

#imprime o resultado da tabela distCapitais.csv
df_distancia

,Cidade,Aracaju,Belém,Belo Horizonte,Boa Vista,Brasília,Campo Grande,Cuiabá,Curitiba,Florianópolis,Fortaleza,Goiânia,João Pessoa,Maceió,Manaus,Natal,Palmas,Porto Alegre,Porto Velho,Recife,Rio Branco,Rio de Janeiro,Salvador,São Luis,São Paulo,Teresina,Vitória
0,Aracaju,0,1641,1248,3022,1292,2155,2121,2061,2207,815,1461,486,201,2673,604,1235,2580,2946,398,3359,1482,277,1226,1731,903,1102
1,Belém,2079,0,2111,1432,1592,2212,1778,2665,2904,1133,1693,1636,1680,1292,1550,973,3188,1886,1676,2333,2450,1687,481,2463,750,2275
2,Belo Horizonte,1578,2824,0,3117,624,1118,1372,820,973,1893,666,1726,1439,2556,1831,1178,1341,2477,1639,2786,339,964,1932,489,1652,378
3,Boa Vista,6000,6083,4736,0,2496,2667,2107,3370,3620,2562,2503,3067,3089,661,2983,1988,3785,1335,3103,1626,3428,3009,1913,3300,2169,3394
4,Brasília,1652,2120,716,4275,0,878,873,1081,1314,1687,173,1716,1485,1932,1775,620,1619,1900,1657,2246,933,1060,1524,873,1313,947
5,Campo Grande,2765,2942,1453,3836,1134,0,559,780,1007,2547,705,2593,2352,2013,2654,1320,1119,1634,2530,1827,1212,1905,2284,894,2132,1490
6,Cuiabá,2775,2941,1594,3142,1133,694,0,1302,1543,2329,740,2495,2302,1453,2524,1029,1679,1137,2452,1414,1575,1915,1942,1326,1862,1745
7,Curitiba,2595,3193,1004,4821,1366,991,1679,0,251,2670,972,2545,2259,2734,2645,1693,546,2412,2459,2601,675,1784,2599,338,2362,1076
8,Florianópolis,2892,3500,1301,5128,1673,1298,1986,300,0,2857,1215,2693,2402,2981,2802,1931,376,2641,2603,2809,748,1930,2821,489,2573,1160
9,Fortaleza,1183,1610,2528,6548,2200,3407,3406,3541,3838,0,1854,555,730,2383,435,1300,3213,2855,629,3300,2190,1028,652,2368,495,1855


In [ ]:
df_distancia.Aracaju[1] #teste

'2079'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls

conexCapitais.csv  distCapitais.csv  sample_data


In [ ]:
#funcoes suporte

#vai marcar o tempo
def marcar_tempo():
  return time.time();

#seleciona a cidade
def escolher_cidade(texto):
  while True:
    cidade = input('Escolha a cidade onde voce quer {}\n'.format(texto))
    correto = valida_cidade(cidade, df_distancia.Cidade)
    if (correto == True):
      return cidade

def inicializando_algoritmo():
  while True:
    algoritmo_escolhido = int(input('Qual algoritmo voce quer executar?\n0 - Busca em largura\n1 - Busca em profundidade\n2 - Busca em profundidade limitada\n3 - Busca em aprofundamento iterativo\n4 - Busca de custo uniforme\n'))    
    if (algoritmo_escolhido < 0 or algoritmo_escolhido > 4):
      print('Algoritmo escolhido invalido, nao temos essa opcao')
    else:
      break
  
  cidade_inicio = escolher_cidade('começar')
  cidade_fim = escolher_cidade('chegar')
  
  if(algoritmo_escolhido == 0):
    print('Algoritmo Busca em largura escolhido')
    algoritmo_busca_em_largura(cidade_inicio, cidade_fim)
  elif(algoritmo_escolhido == 1):
    print('Algoritmo Busca em profundidade escolhido')
    algoritmo_busca_em_profundidade(cidade_inicio, cidade_fim)
  elif(algoritmo_escolhido == 2):
    limite = int(input('Escolha um limite de nós: '))
    print('Algoritmo Busca em profundidade limitada escolhido')
    algoritmo_busca_em_profundidade_limitada(cidade_inicio, cidade_fim, limite)
  elif(algoritmo_escolhido == 3):
    print('Algoritmo Busca em aprofundamento iterativo escolhido')
    busca_em_profundidade_interativa(cidade_inicio, cidade_fim)
  elif(algoritmo_escolhido == 4):
    print('Algoritmo Busca de custo uniforme escolhido')
    algoritmo_busca_em_largura(cidade_inicio, cidade_fim)
  
#valida a cidade se existe em nossa tabela ou nao
def valida_cidade(cidade, todas_as_cidades):
  for cidade_aux in todas_as_cidades:
    if (cidade == cidade_aux):
      return True
  
  print('Nao encontrado a cidade. Tente novamente')
  return False

In [ ]:
#execucao do algoritmo de busca em largura
def algoritmo_busca_em_largura(cidade_inicio, cidade_fim):
  tempo_inicial = marcar_tempo()
  resultado_algoritmo = bfs(cidade_inicio, cidade_fim)  
  tempo_final = marcar_tempo()
  
  if resultado_algoritmo != -1:
    print('A menor distancia: ', resultado_algoritmo)
  
  print('O tempo percorrido: ', tempo_final - tempo_inicial)

#algoritmo bfs
def bfs(cidade_inicio, cidade_fim):
  visitados = set() #declara uma variavel do tipo set
  fila = [] #cria uma lista
  fila.append([0, cidade_inicio]) #adiciona a cidade selecionada como primeira da fila

  while len(fila) != 0:
    distancia, cidade = fila.pop(0) #pega a distancia já calculado e a cidade onde estamos atualmente 
    
    if(cidade == cidade_fim): #achou a cidade destino
      return distancia

    if cidade not in visitados: #cidade nao visitada ainda
      visitados.add(cidade)
      for i, conexao in enumerate(df_conexao[cidade]): 
        if conexao == 1: #verifica se ha conexao
          cidade_viz = df_distancia.Cidade[i]
          
          if cidade_viz in visitados: #cidade vizinha ja visitada
            continue
          
          fila.append([distancia + int(df_distancia[cidade][i]), cidade_viz]) #adiciona a distancia da cidade atual + vizinha e a cidade vizinha

    fila.sort(key=lambda x:x[0]) #ordena a fila, sempre queremos a menor distancia possivel

  print("Nao encontramos essa cidade selecionada")
  return -1


In [ ]:
#execucao do algoritmo de busca em profundidade
def algoritmo_busca_em_profundidade(cidade_inicio, cidade_fim):
  tempo_inicial = marcar_tempo()
  resultado_algoritmo = dfs(cidade_inicio, cidade_fim)  
  tempo_final = marcar_tempo()
  
  if resultado_algoritmo != -1:
    print('A menor distancia: ', resultado_algoritmo)
  
  print('O tempo percorrido: ', tempo_final - tempo_inicial)

#algoritmo dfs
def dfs(cidade_inicio, cidade_fim):
  visitados = set()
  
  return dfs_recursivo(visitados, 0, cidade_inicio, cidade_fim)

#algoritmo dfs recursivo, so um suporte
def dfs_recursivo(visitados, distancia, cidade_atual, cidade_fim):
    if (cidade_atual == cidade_fim):
      return distancia 

    visitados.add(cidade_atual) #cidade atual adicionada como visitada
    res = -1
    
    for indice, conexao in enumerate(df_conexao[cidade_atual]):
      if conexao == 1: #verifica se tem conexao
        cidade_viz = df_distancia.Cidade[indice] #pega cidade vizinha
            
        if cidade_viz in visitados: #verifica se cidade vizinha foi visitada
          continue
        
        dist = distancia + int(df_distancia[cidade_atual][indice]) #distancia entre distancia até a cidade que selecionamos + distancia entre cidade atual e vizinha
        
        if (res != -1):
          valor_dfs = dfs_recursivo(visitados, dist, cidade_viz, cidade_fim)
          
          if valor_dfs != -1: #evita que consideramos o valor == -1 na funçao
            res = min(res, valor_dfs)            
        else:
          res = dfs_recursivo(visitados, dist, cidade_viz, cidade_fim)          
    
    visitados.remove(cidade_atual) #remove cidade como visitado
    return res


In [ ]:
def algoritmo_busca_em_profundidade_limitada(cidade_inicio, cidade_fim,limite):
  tempo_inicial = marcar_tempo()
  resultado_algoritmo = dfs_limitado(cidade_inicio, cidade_fim,limite)  
  tempo_final = marcar_tempo()

  if resultado_algoritmo != -1:
    print('A menor distancia: ', resultado_algoritmo)
  else:
     print("Não foi identificada uma rota até o destino")

  print('O tempo percorrido: ', tempo_final - tempo_inicial)

#algoritmo dfs
def dfs_limitado(cidade_inicio, cidade_fim, limite):
  visitados = set()

  return dfs_recursivo_limitado(visitados, 0, cidade_inicio, cidade_fim, limite)

#algoritmo dfs recursivo, so um suporte
def dfs_recursivo_limitado(visitados, distancia, cidade_atual, cidade_fim, limite):
    if (cidade_atual == cidade_fim):
       return distancia 

    visitados.add(cidade_atual) #cidade atual adicionada como visitada
    res = -1
    
    if (limite > 0): #Define o limite de execuções do algoritmo, como o limite é decrementado a cada nova chamada da função, a profundidade máxima da busca irá parar quando o limite chegar a 0
      for indice, conexao in enumerate(df_conexao[cidade_atual]):
        if conexao == 1: #verifica se tem conexao
          cidade_viz = df_distancia.Cidade[indice] #pega cidade vizinha
          
          if cidade_viz in visitados: #verifica se cidade vizinha foi visitada
            continue

          #distancia entre distancia até a cidade que selecionamos + distancia entre cidade atual e vizinha
          dist = distancia + int(df_distancia[cidade_atual][indice])
          
          if (res != -1):
           valor_dfs = dfs_recursivo_limitado(visitados, dist, cidade_viz, cidade_fim, limite - 1) #Limite decrementa para limitar a profundidade do algoritmo

           if valor_dfs != -1: #evita que consideramos o valor == -1 na funçao
              res = min(res, valor_dfs)            
          else:
            res = dfs_recursivo_limitado(visitados, dist, cidade_viz, cidade_fim, limite - 1) #Limite decrementa para limitar a profundidade do algoritmo
    
    visitados.remove(cidade_atual) #remove cidade como visitado é preciso remover para que a lista seja limpa
    #o laço chegar ao fim se não encontrar o resultado entre os filhos do nó, neste caso será necessário limpar o caminho e partir para o novo caminho
    
    return res

In [ ]:
#Busca em profundidade iterativa
def busca_em_profundidade_interativa(cidade_inicio, cidade_fim):
  i = 0
  resultado_dfs = -1

  while (resultado_dfs == -1): #Enquanto a função de busca em profundidade não retornar um resultado, será inserido um nó a mais de profundidade
    resultado_dfs = dfs_limitado(cidade_inicio, cidade_fim, i)
    i= i + 1

  print("A menor distancia: ", resultado_dfs)
  print("Profundidade do nó objetivo: {}".format(i-1))
  


In [ ]:
inicializando_algoritmo()

Qual algoritmo voce quer executar?
0 - Busca em largura
1 - Busca em profundidade
2 - Busca em profundidade limitada
3 - Busca em aprofundamento iterativo
4 - Busca de custo uniforme
3
Escolha a cidade onde voce quer começar
Aracaju
Escolha a cidade onde voce quer chegar
Brasília
Algoritmo Busca em aprofundamento iterativo escolhido
A menor distancia:  1416
Profundidade do nó objetivo: 2
